In [604]:
from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
import pandas as pd
import torch
from tqdm.auto import trange
torch.cuda.is_available()

True

In [605]:
load_dotenv()

True

In [606]:
papers = pd.read_excel("./Search String H Results.xlsx")
print(len(papers))

1626


In [607]:
categories = [
    "Oceanography",
]
vectorstore = FAISS.from_embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs=dict(device="cuda:0"))

category_embeddings = embeddings.embed_documents(categories)
db = FAISS.from_embeddings(list(zip(categories, category_embeddings)), embeddings)

In [608]:
oceanography_score = []
for i in trange(len(papers)):
    text = papers.loc[i]['Title'] + "\n\n" + papers.loc[i]['Abstract']

    top_4 = db.similarity_search_with_score(text, k=1)
    label = [score for doc, score in top_4][0]
    oceanography_score.append(label)

papers['oceanography_score'] = oceanography_score.copy()
    

  0%|          | 0/1626 [00:00<?, ?it/s]

In [609]:
top_4

[(Document(page_content='Oceanography'), 1.1977897)]

In [621]:
categories = [
    "Whale identification",
    "Microplastics",
    "Erosion",
    "Ocean circulation and currents",
    "Climate modelling",
    "Detection of man-made objects",
    "Algae and harmful algae bloom detection",
    "Regime dynamics",
    "Species distribution",
    "Shipping, navigation, and logistics",
    "Water level prediciton",
    "Infrastructure monitoring and corrosion detection",
    "Eelgrass monitoring",
    "Contaminated Ocean Water detection",
    "Fleet monitoring",
    "Acoustic classification",
    "Biodiversity monitoring",
    "Plankton identification",
    "Invasive species monitoring",
    "Seaweed monitoring",
    "Disease detection",
    "Natural disaster forecasting",
    "Engineering",
    "Coastal geomorphology",
    "Ocean chemistry detection",
    "Underwater photography",
    "Climate engineering",
    "Fishing and seafood",
    "Power generation",
    "Resource management",
    "Unmanned vehicles",
    "Ice monitoring",
    "Sea floor mapping and bathymetry",
    "Marine ecology",
    "Coral reef monitoring and bleaching",
    "Taxonomy",
]

vectorstore = FAISS.from_embeddings

# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs=dict(device="cuda:0"))
embeddings = OpenAIEmbeddings()

category_embeddings = embeddings.embed_documents(categories)
db = FAISS.from_embeddings(list(zip(categories, category_embeddings)), embeddings)

In [622]:
for i in trange(len(papers)):
    if papers.iloc[i]['oceanography_score'] > 1.1:
        continue
        
    text = papers.iloc[i]['Title'] + "\n\n" + papers.iloc[i]['Abstract']
    top_4 = db.similarity_search_with_score(text)
    
    if top_4[0][1] > 1:
        print(i)
        print(papers.iloc[i]['oceanography_score'])
        print(text)
        print()
        for t in top_4:
            print(t)
        print()


  0%|          | 0/1626 [00:00<?, ?it/s]

In [623]:
labels = []
for i in trange(len(papers)):
    text = papers.loc[i]['Title'] + "\n\n" + papers.loc[i]['Abstract']

    top_4 = db.similarity_search_with_score(text)
    label = "; ".join([f"{doc.page_content} ({score.round(3):.3f})" for doc, score in top_4])
    labels.append(label)

  0%|          | 0/1626 [00:00<?, ?it/s]

In [624]:
papers['Auto Application'] = labels

In [625]:
papers.to_excel("topics.xlsx")

In [626]:
for c in categories:
    print(c)

Whale identification
Microplastics
Erosion
Ocean circulation and currents
Climate modelling
Detection of man-made objects
Algae and harmful algae bloom detection
Regime dynamics
Species distribution
Shipping, navigation, and logistics
Water level prediciton
Infrastructure monitoring and corrosion detection
Eelgrass monitoring
Contaminated Ocean Water detection
Fleet monitoring
Acoustic classification
Biodiversity monitoring
Plankton identification
Invasive species monitoring
Seaweed monitoring
Disease detection
Natural disaster forecasting
Engineering
Coastal geomorphology
Ocean chemistry detection
Underwater photography
Climate engineering
Fishing and seafood
Power generation
Resource management
Unmanned vehicles
Ice monitoring
Sea floor mapping and bathymetry
Marine ecology
Coral reef monitoring and bleaching
Taxonomy


In [630]:
papers['abstract_words'] = papers['Abstract'].str.split().str.len()

In [631]:
papers['title_words'] = papers['Title'].str.split().str.len()

In [636]:
papers['abstract_words'].sum()

395112

In [637]:
papers['title_words'].sum()

22458